In [1]:
import numpy as np
import pandas as pd

import gurobipy as grb
from gurobipy import GRB

## Problem 1

In [2]:
# define the data & limitations
clothing = {"Sweaters": {'Sale': 75, 'Cost': 20, 'Labor': 1.2, 'Cloth': 2}, 
            "Vests": {'Sale': 60, 'Cost': 15, 'Labor': 1, 'Cloth': 1}}
total_labor = 600
total_cloth = 900

# create a generic model to interact
m1 = grb.Model("question_1")

# ---------------------
# DECISION VARIABLES
# ---------------------
# create variables
alloc = {}
for c in clothing.keys():
    alloc[c] = m1.addVar(vtype=GRB.CONTINUOUS, name=f"alloc_{c}")

# ---------------------
# OBJECTIVE FUNCTION
# ---------------------
# Set objective function, maximize the profit of the given dynamic 
m1.setObjective(sum([ (clothing[c]['Sale']-clothing[c]['Cost']) * alloc[c] for c in clothing]), GRB.MAXIMIZE)

# ---------------------
# CONSTRAINTS
# ---------------------
# add universal constraints i.e. total_labor
m1.addConstr(sum([clothing[c]['Labor'] * alloc[c] for c in clothing]) == total_labor , "total_labor")
m1.addConstr(sum([clothing[c]['Cloth'] * alloc[c] for c in clothing]) == total_cloth , "total_cloth")

# add individual (case specific) constraints e.g. minimum vaccine reqs
for c in clothing.keys():
    m1.addConstr(alloc[c] >= 0 , f"min_alloc_{c}")

# ---------------------
# OPTIMIZE
# ---------------------
# call method to ptimize model
m1.optimize()

# call the .x parameter to return allocation solution  
for c in clothing.keys(): 
    print(f'Allocation to Station {c}: {alloc[c].x}')

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 4 rows, 2 columns and 6 nonzeros
Model fingerprint: 0xf2ea3968
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [5e+01, 6e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+02, 9e+02]
Presolve removed 4 rows and 2 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.7375000e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.737500000e+04
Allocation to Station Sweaters: 375.0
Allocation to Station Vests: 150.0


## Problem 2

**1. If a linear programming problem has a feasible solution, it is guaranteed to have an optimal solution.**

False, if a feasible solution exists then it satisfies all the constraints of the problem. However, if unbounded the region may not have an optimal solution. 

**2. If the feasible region of a linear programming problem is unbounded, it means the objective function can be improved indefinitely.**

True, if unbounded there exists no optimal solution to the feasible region bounded by constraints, so you can move indefinitely in the region. 

**3. If an LP is feasible, then it has an optimal solution that is a corner point.**

True, if a linear-program has a feasible range, then the optimal solution will exist within a corner point as they represent extremes within the constraints.  

**4. The graphical solution method can only be used to solve linear programs with two decision variables.**

False, the graphical solution plots linear relations on a graph and can be used to display both two and three decision variable problems.


## Problem 3

In [3]:
# Data: Supplier information and cost per unit (c_ijk)
suppliers = ['Supplier1', 'Supplier2', 'Supplier3']
resources = ['Shelter', 'Medical Care', 'Legal Aid']
intensities = ['Low', 'Medium', 'High']

# Placeholder cost values for each combination
cost_per_unit = {
    ('Supplier1', 'Shelter', 'Low'): 100,
    ('Supplier1', 'Shelter', 'Medium'): 120,
    ('Supplier1', 'Shelter', 'High'): 150,
    ('Supplier1', 'Medical Care', 'Low'): 80,
    ('Supplier1', 'Medical Care', 'Medium'): 100,
    ('Supplier1', 'Medical Care', 'High'): 130,
    ('Supplier1', 'Legal Aid', 'Low'): 90,
    ('Supplier1', 'Legal Aid', 'Medium'): 110,
    ('Supplier1', 'Legal Aid', 'High'): 140,
    ('Supplier2', 'Shelter', 'Low'): 130,
    ('Supplier2', 'Shelter', 'Medium'): 140,
    ('Supplier2', 'Shelter', 'High'): 160,
    ('Supplier2', 'Medical Care', 'Low'): 50,
    ('Supplier2', 'Medical Care', 'Medium'): 70,
    ('Supplier2', 'Medical Care', 'High'): 100,
    ('Supplier2', 'Legal Aid', 'Low'): 120,
    ('Supplier2', 'Legal Aid', 'Medium'): 140,
    ('Supplier2', 'Legal Aid', 'High'): 150,
    ('Supplier3', 'Shelter', 'Low'): 80,
    ('Supplier3', 'Shelter', 'Medium'): 110,
    ('Supplier3', 'Shelter', 'High'): 150,
    ('Supplier3', 'Medical Care', 'Low'): 90,
    ('Supplier3', 'Medical Care', 'Medium'): 120,
    ('Supplier3', 'Medical Care', 'High'): 140,
    ('Supplier3', 'Legal Aid', 'Low'): 70,
    ('Supplier3', 'Legal Aid', 'Medium'): 110,
    ('Supplier3', 'Legal Aid', 'High'): 130,
}

# Minimum required quantities as a dataset
minimum_quantities = {
    'Shelter': {'Low': 100, 'Medium': 300, 'High': 100} ,
    'Medical Care': {'Low': 50, 'Medium': 150, 'High': 100} ,
    'Legal Aid': {'Low': 50, 'Medium': 100, 'High': 150}
}

In [18]:
# create a generic model to interact
m2 = grb.Model("question_3")

# ---------------------
# DECISION VARIABLES
# ---------------------
# create quantity variables
alloc = {}
for c in cost_per_unit.keys():
    alloc[c] = m2.addVar(vtype=GRB.CONTINUOUS, name=f"alloc_{c}")

# ---------------------
# OBJECTIVE FUNCTION
# ---------------------
# Set objective function, maximize the profit of the given dynamic 
m2.setObjective(sum([cost_per_unit[c]  * alloc[c] for c in cost_per_unit]), 
                GRB.MINIMIZE)

# ---------------------
# CONSTRAINTS
# ---------------------
# add constraint for the minimum resource constraint required
for i in resources:
    for j in intensities:
        m2.addConstr(sum([alloc[(k,i,j)] for k in suppliers]) >= minimum_quantities[i][j], f'min_{i}_{j}')

# add constraint for each supplier limited to 500 resources
for k in suppliers:
    m2.addConstr(sum([alloc[(k,i,j)] for i in resources for j in intensities]) <= 500, f'total_{k}_limit')

# add non-negative constraint for resoures
for c in cost_per_unit.keys():
    m2.addConstr(alloc[c] >= 0, f'non_negative_{c}')

# ---------------------
# OPTIMIZE
# ---------------------
# call method to ptimize model
m2.optimize()

est = {}
# call the .x parameter to return allocation solution  
for c in cost_per_unit.keys(): 
    est[c] =  alloc[c].x

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 39 rows, 27 columns and 81 nonzeros
Model fingerprint: 0x7f0f4b5d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e+01, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 5e+02]
Presolve removed 27 rows and 0 columns
Presolve time: 0.00s
Presolved: 12 rows, 27 columns, 54 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.375000e+02   0.000000e+00      0s
      10    1.1400000e+05   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.140000000e+05


In [19]:
out_df = pd.DataFrame.from_dict(est, columns=['Allocation'], orient='index').reset_index()
lhs = pd.DataFrame(out_df['index'].apply(lambda x: list(x)).tolist(), 
                   columns=['Supplier', 'Resource', 'Intensity'])
results = pd.concat([lhs, out_df['Allocation']], axis=1) 

In [20]:
pd.pivot_table(data=results, index=['Resource', 'Intensity'], columns='Supplier', values='Allocation')

Supplier                Supplier1  Supplier2  Supplier3
Resource     Intensity                                 
Legal Aid    High             0.0        0.0      150.0
             Low              0.0        0.0       50.0
             Medium         100.0        0.0        0.0
Medical Care High             0.0      100.0        0.0
             Low              0.0       50.0        0.0
             Medium           0.0      150.0        0.0
Shelter      High           100.0        0.0        0.0
             Low              0.0        0.0      100.0
             Medium         100.0        0.0      200.0

## Problem 4

In [59]:
# Define Workers , Departments , Shifts , and Days
workers = [i for i in range(1, 101)]
departments = ['Battery', 'Body', 'Assembly', 'Paint', 'Quality']
shifts = ['Morning', 'Afternoon', 'Night']
days = ['Mon', 'Tue', 'Wed', 'Thur', 'Fri', 'Sat', 'Sun']

# Create Workers DataFrame
workers_df = pd. DataFrame ({
    'Worker_ID': np.repeat(workers , len( departments )*len(shifts)*len(days)),
    'Department': np.tile(np.repeat(departments , len(shifts)*len(days)), len(workers)),
    'Shift': np.tile(np.repeat(shifts , len(days)), len(workers)*len(departments )),
    'Day': np.tile(days , len(workers)*len( departments )*len(shifts)),
    'Availability': np.random.choice ([0, 1], len(workers)*len( departments )*len(shifts)*len(days)),
    'Preference_Score': np.random.randint (1, 10, len(workers)*len(departments )*len(shifts)*len(days)),
    'Effectiveness_Score': np.random.randint (1, 10, len(workers)*len(departments )*len(shifts)*len(days))
})

# Create Department DataFrame
dept_df = pd. DataFrame ({
    'Department': np.repeat(departments , len(shifts)*len(days)),
    'Shift': np.tile(np.repeat(shifts , len(days)), len( departments )),
    'Day': np.tile(days , len( departments )*len(shifts)),
    'Min_Workers': np.random.randint (1, 5, len( departments )*len(shifts)*len(days)),
    'Max_Workers': np.random.randint (5, 10, len( departments )*len(shifts)*len(days))
})

In [60]:
# remap the worker and dept df to aid in lookup 
workers_dict = workers_df.set_index(['Worker_ID', 'Department', 'Shift', 'Day']).to_dict()
dept_dict = dept_df.set_index(['Department', 'Shift', 'Day']).to_dict()

In [64]:
# create a generic model to interact
m3 = grb.Model("question_4")

# ---------------------
# DECISION VARIABLES
# ---------------------
# create binary variable to determine if someone worked for a given Day-Shift-Department combination
binary_shift = m3.addVars(workers, departments, shifts, days, vtype=GRB.BINARY)

# ---------------------
# OBJECTIVE FUNCTION
# ---------------------
# set objective function, maximize the profit of the given dynamic 
m3.setObjective(sum(workers_dict['Preference_Score'][w, d, s, a]  * 
                    binary_shift[w,d,s,a] *
                    workers_dict['Effectiveness_Score'][w, d, s, a]
                    for w in workers for d in departments for s in shifts for a in days), 
                GRB.MAXIMIZE)

# ---------------------
# CONSTRAINTS
# ---------------------
# add shift constraint, each worker can only work one shift a day 
for w in workers:
    for a in days:
        m3.addConstr(sum(binary_shift[w,d,s,a] for d in departments for s in shifts) <= 1, 
                     f'shift_constr_{w}_{a}_lmt')

# add week constraint, each worker can only work 5 days a week
for w in workers:
    m3.addConstr(sum(binary_shift[w,d,s,a] for d in departments for s in shifts for a in days) <= 5, 
                 f'week_constr_{w}_lmt')

# availability constraint, each worker can only work on days they're available to work
for w in workers:
    for d in departments:
        for s in shifts:
            for a in days:
                m3.addConstr(binary_shift[w,d,s,a] <= workers_dict['Availability'][w, d, s, a], 
                             f'availability_constr_{c}_lmt')

# worker constraint, number of workers must meet the minimum and maximum staffing requirements 
for d in departments:
    for s in shifts:
        for a in days:
            m3.addConstr(sum(binary_shift[w,d,s,a] for w in workers) <= dept_dict['Max_Workers'][d,s,a], 
                         f'max_worker_constr_{d}_{s}_{a}_lmt')
            m3.addConstr(sum(binary_shift[w,d,s,a] for w in workers) >= dept_dict['Min_Workers'][d,s,a], 
                         f'min_worker_constr_{d}_{s}_{a}_lmt')
                
# ---------------------
# OPTIMIZE
# ---------------------  
# call method to ptimize model
m3.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 11510 rows, 10500 columns and 52500 nonzeros
Model fingerprint: 0x90746154
Variable types: 0 continuous, 10500 integer (10500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+00]
Found heuristic solution: objective 13082.000000
Presolve removed 10500 rows and 5277 columns
Presolve time: 0.03s
Presolved: 1010 rows, 5223 columns, 20892 nonzeros
Variable types: 0 continuous, 5223 integer (5223 binary)
Found heuristic solution: objective 27662.000000

Root relaxation: objective 3.070200e+04, 277 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |

In [65]:
out_df = pd.DataFrame.from_dict(binary_shift, columns=['Allocation'], orient='index').reset_index()
lhs = pd.DataFrame(out_df['index'].apply(lambda x: list(x)).tolist(), 
                   columns=['Worker_ID', 'Department', 'Shift', 'Day'])
results = pd.concat([lhs, out_df['Allocation']], axis=1) 
results['Allocation'] = results['Allocation'].apply(lambda x: x.x) 
results = pd.merge(results, workers_df, on=['Worker_ID', 'Department', 'Shift', 'Day'])

In [66]:
# display the worker schedules i.e. the value 
schedule_results = results[results['Allocation'] == 1]

In [67]:
schedule_results

,Worker_ID,Department,Shift,Day,Allocation,Availability,Preference_Score,Effectiveness_Score
10,1,Battery,Afternoon,Thur,1.0,1,9,6
47,1,Assembly,Morning,Sat,1.0,1,8,6
56,1,Assembly,Night,Mon,1.0,1,7,9
74,1,Paint,Afternoon,Fri,1.0,1,8,9
99,1,Quality,Night,Tue,1.0,1,7,9
...,...,...,...,...,...,...,...,...
10422,100,Body,Morning,Sun,1.0,1,9,7
10432,100,Body,Night,Wed,1.0,1,7,9
10475,100,Paint,Night,Thur,1.0,1,8,9
10476,100,Paint,Night,Fri,1.0,1,8,8


In [68]:
# example schedule a for a worker given constraints 
# 1) each worker can work one shift a day
# 2) each worker can work a maxium of five days a week 
# 3) each worker can only work on days they are available 

schedule_results[schedule_results['Worker_ID'] == 1]

,Worker_ID,Department,Shift,Day,Allocation,Availability,Preference_Score,Effectiveness_Score
10,1,Battery,Afternoon,Thur,1.0,1,9,6
47,1,Assembly,Morning,Sat,1.0,1,8,6
56,1,Assembly,Night,Mon,1.0,1,7,9
74,1,Paint,Afternoon,Fri,1.0,1,8,9
99,1,Quality,Night,Tue,1.0,1,7,9


### Sensitivity Analysis & Duality Theorem

In [ ]:
# Sensitivity Analysis
print("\nSensitivity Analysis")
    
# Shadow Prices or Dual Values
print("\nShadow Prices: how much the objective function would change if you increase the right-hand side of the constraint by one unit") # Shadow Prices can be obtained using c.Pi where c is a constraint.
for c in m.getConstrs():
    print(f"{c.ConstrName}: {c.Pi}")
        
# Reduced Costs  
print("\nReduced Costs: how much the objective function would change if a decision variable is increased by one unit")  # Reduced Costs can be found using v.RC where v is a variable.
for v in m.getVars():
    print(f"{v.VarName}: {v.RC}")
    
# Ranges in which the current basis remains optimal
print("\nRanges for RHS: how much you could change the constraint without changing the optimal basis") # Ranges for RHS within which the current basis stays optimal are found using c.SARHSLow and c.SARHSUp where c is a constraint.
for c in m.getConstrs():
    print(f"{c.ConstrName}: {c.SARHSLow} to {c.SARHSUp}")
        
print("\nRanges for Objective Coefficients: how much you could change the objective coefficient before changing the optimal basis")
for v in m.getVars():     # Ranges for Objective Coefficients within which the current basis stays optimal are found using v.SAObjLow and v.SAObjUp where v is a variable.
    print(f"{v.VarName}: {v.SAObjLow} to {v.SAObjUp}")